In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [ ]:
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', engineSource)

mensajeria_estadosservicio.drop(columns=["foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)


def actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, date1, time1, date2, time2, duration):

    if (biblioteca_de_servicios_y_registros[servicio_id].get(date2) is None or 
        biblioteca_de_servicios_y_registros[servicio_id].get(time2) is None):
        
        assignment_date = mensajeria_estadosservicio.iloc[index, 1].strftime("%Y-%m-%d")
        assignment_time = mensajeria_estadosservicio.iloc[index, 2].strftime('%H:%M')
        
        biblioteca_de_servicios_y_registros[servicio_id][date2] = assignment_date
        biblioteca_de_servicios_y_registros[servicio_id][time2] = assignment_time
        
        if (biblioteca_de_servicios_y_registros[servicio_id].get(date1) is not None and 
            biblioteca_de_servicios_y_registros[servicio_id].get(time1) is not None):

            start_date = biblioteca_de_servicios_y_registros[servicio_id][date1]
            start_time = biblioteca_de_servicios_y_registros[servicio_id][time1]
            
            start_datetime = datetime.strptime(f"{start_date} {start_time}", "%Y-%m-%d %H:%M")
            assignment_datetime = datetime.strptime(f"{assignment_date} {assignment_time}", "%Y-%m-%d %H:%M")
            
            biblioteca_de_servicios_y_registros[servicio_id][duration] = assignment_datetime - start_datetime
            
        else:
            print(">", servicio_id)


biblioteca_de_servicios_y_registros = {}

for index, row in mensajeria_estadosservicio.iterrows():
    servicio_id = mensajeria_estadosservicio.iloc[index, 4]
    estado_id = mensajeria_estadosservicio.iloc[index, 3]

    if servicio_id not in biblioteca_de_servicios_y_registros:
        biblioteca_de_servicios_y_registros[servicio_id] = {'key_service': servicio_id, 
                                                            'key_start_date': None, 
                                                            'key_start_time': None, 
                                                            'key_assignment_date': None, 
                                                            'key_assignment_time': None, 
                                                            'key_pick_up_date': None, 
                                                            'key_pick_up_time': None, 
                                                            'key_delivery_date': None, 
                                                            'key_delivery_time': None, 
                                                            'key_closing_date': None, 
                                                            'key_closing_time': None,
                                                            'start_to_assignment_duration': None,
                                                            'assignment_to_pick_up_duration': None,
                                                            'pick_up_to_delivery_duration': None,
                                                            'delivery_to_closing_duration': None,}
        
        # if estado_id == 1:
        #     biblioteca_de_servicios_y_registros[servicio_id]['key_start_date'] = mensajeria_estadosservicio.iloc[index, 1]
        #     biblioteca_de_servicios_y_registros[servicio_id]['key_start_time'] = mensajeria_estadosservicio.iloc[index, 2].strftime('%H:%M')

    #     elif estado_id == 2:
    #         actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_start_date', 'key_start_time', 'key_assignment_date', 'key_assignment_time')

    #     elif estado_id == 4:
    #         actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_assignment_date', 'key_assignment_time', 'key_pick_up_date', 'key_pick_up_time')

    #     elif estado_id == 5:
    #         actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_pick_up_date', 'key_pick_up_time', 'key_delivery_date', 'key_delivery_time')

    #     elif estado_id == 6:
    #         actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_delivery_date', 'key_delivery_time', 'key_closing_date', 'key_closing_time')

        
    if estado_id == 1:
        biblioteca_de_servicios_y_registros[servicio_id]['key_start_date'] = mensajeria_estadosservicio.iloc[index, 1].strftime("%Y-%m-%d")
        biblioteca_de_servicios_y_registros[servicio_id]['key_start_time'] = mensajeria_estadosservicio.iloc[index, 2].strftime('%H:%M')

    elif estado_id == 2:
        actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_start_date', 'key_start_time', 'key_assignment_date', 'key_assignment_time', 'start_to_assignment_duration')

    elif estado_id == 4:
        actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_assignment_date', 'key_assignment_time', 'key_pick_up_date', 'key_pick_up_time', 'assignment_to_pick_up_duration')

    elif estado_id == 5:
        actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_pick_up_date', 'key_pick_up_time', 'key_delivery_date', 'key_delivery_time', 'pick_up_to_delivery_duration')

    elif estado_id == 6:
        actualizar_registro(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_delivery_date', 'key_delivery_time', 'key_closing_date', 'key_closing_time', 'delivery_to_closing_duration')



fact_messaging = pd.DataFrame.from_dict(biblioteca_de_servicios_y_registros, orient='index')



# # # # # #
# dim_date = pd.read_sql_table('dim_date', engineDestination)
# dim_time = pd.read_sql_table('dim_time', engineDestination)
# # # # # #

fact_messaging

# print(a[0], type(a[0]))
# print(a[10], type(a[10]))
# print(a[100], type(a[100]))



In [ ]:
count_none = fact_messaging['key_start_date'].isnull().sum()
print("key_start_date:", count_none)

count_none = fact_messaging['key_start_time'].isnull().sum()
print("key_start_time:", count_none)

count_none = fact_messaging['key_assignment_date'].isnull().sum()
print("key_assignment_date:", count_none)

count_none = fact_messaging['key_assignment_time'].isnull().sum()
print("key_assignment_time:", count_none)

count_none = fact_messaging['key_pick_up_date'].isnull().sum()
print("key_pick_up_date:", count_none)

count_none = fact_messaging['key_pick_up_time'].isnull().sum()
print("key_pick_up_time:", count_none)

count_none = fact_messaging['key_delivery_date'].isnull().sum()
print("key_delivery_date:", count_none)

count_none = fact_messaging['key_delivery_time'].isnull().sum()
print("key_delivery_time:", count_none)

count_none = fact_messaging['key_closing_date'].isnull().sum()
print("key_closing_date:", count_none)

count_none = fact_messaging['key_closing_time'].isnull().sum()
print("key_closing_time:", count_none)


print("-------------")


count_none = fact_messaging['start_to_assignment_duration'].isnull().sum()
print("start_to_assignment_duration:", count_none)

count_none = fact_messaging['assignment_to_pick_up_duration'].isnull().sum()
print("assignment_to_pick_up_duration:", count_none)

count_none = fact_messaging['pick_up_to_delivery_duration'].isnull().sum()
print("pick_up_to_delivery_duration:", count_none)

count_none = fact_messaging['delivery_to_closing_duration'].isnull().sum()
print("delivery_to_closing_duration:", count_none)
			
# count_nat = fact_messaging['key_pick_up_time'].isna().sum() - count_none
# print("Número de NaT en la columna:", count_nat)